# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from pathlib import Path
import os
import pandas as pd
import numpy as np
import nltk
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [2]:
url = "https://raw.githubusercontent.com/vanerven/md-trabalho-final/master/dados/csv/SistemaDeClassificacao.csv"
classes_original = pd.read_csv(url, sep=',', encoding='ISO-8859-1')
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [3]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [4]:
url = "https://raw.githubusercontent.com/vanerven/md-trabalho-final/master/dados/csv/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.csv"
leis_classificadas_original = pd.read_csv(url, sep=',', encoding='ISO-8859-1')
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as transformações iniciais.

In [5]:
url = "https://raw.githubusercontent.com/vanerven/md-trabalho-final/master/dados/csv/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.csv"
leis_original = pd.read_csv(url, sep=',', encoding='ISO-8859-1')
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set..."
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...


In [6]:
leis_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [7]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente


In [8]:
leis.shape[0]

27743

Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [9]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
20,36062349,33808912.0,Política Social
21,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
27349,36032872,33808912.0,Política Social
27350,35556312,33685789.0,Administração Pública
27351,35556312,33808912.0,Política Social
27359,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de leis.

In [10]:
leis.shape[0]

27294

Total de classificações feitas em leis.

In [11]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()", engine="python")
quantidade_classificacoes = leis_classificadas.shape[0]
quantidade_classificacoes

17438

Número distinto de leis classificadas.

In [12]:
len(leis_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de leis a classificar.

In [13]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()", engine="python")
leis_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [14]:
distribuicao_classes = leis_classificadas[["COD_CLASSE", "DES_CLASSE"]].groupby("DES_CLASSE").count() / quantidade_classificacoes * 100
distribuicao_classes.sort_values(by="COD_CLASSE", ascending=False)

,COD_CLASSE
DES_CLASSE,
Orçamento Público,64.181672
Infraestrutura,14.382383
Política Social,5.029246
Administração Pública,4.490194
Honorífico,3.847918
Economia e Desenvolvimento,3.693084
Jurídico,2.184884
"Soberania, Defesa Nacional e Ordem Pública",0.951944
Organização do Estado,0.745498


## 2 - Classifiers
***

In [15]:
def classify(classifier_name, classifier, vectorizer, X_smote_treino, y_smote_treino, y_teste, X_tfidf_teste, law_class):
    model = {}

    classifier.fit(X_smote_treino, y_smote_treino)
    predicted_class = classifier.predict(X_tfidf_teste)
    accuracy = f1_score(predicted_class, y_teste)
    print("Algorithm:", classifier_name, "F1-Score:", round(accuracy, 2))
    cr = classification_report(y_teste, predicted_class)
    model = (law_class, accuracy, classifier, vectorizer, classifier_name)
    print(cr)

    return model

In [16]:
def model(law_class):
    
    X = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"])
    y = leis_classificadas["DES_CLASSE"].apply(lambda label: 1 if label.lower() == law_class.lower() else 0)
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    vectorizer = TfidfVectorizer(stop_words=stopwords.words('portuguese'), ngram_range = (1,2))
    vectorizer.fit(X_treino['TXT_EMENTA'])
    X_tfidf_treino = vectorizer.transform(X_treino['TXT_EMENTA'])
    X_tfidf_teste = vectorizer.transform(X_teste['TXT_EMENTA'])

    oversample = SMOTE()
    X_smote_treino, y_smote_treino = oversample.fit_resample(X_tfidf_treino, y_treino)

    # Classifier LogisticRegression
    classifier = LogisticRegression(max_iter=3000)
    model = classify("Logistic Regression", classifier, vectorizer, X_smote_treino, y_smote_treino, y_teste, X_tfidf_teste, law_class)
    # Classifier RandomForest
    classifier = RandomForestClassifier(max_depth=50, n_estimators=200, random_state=0, n_jobs=-1)
    model_aux = classify("Random Forest", classifier, vectorizer, X_smote_treino, y_smote_treino, y_teste, X_tfidf_teste, law_class)
    if model_aux[1] > model[1]:
        model = model_aux
    # Classifier Multi-layer Perceptron
    classifier = MLPClassifier(random_state=1, max_iter=200).fit(X_smote_treino, y_smote_treino)
    model_aux = classify("Multi-layer Perceptron", classifier, vectorizer, X_smote_treino, y_smote_treino, y_teste, X_tfidf_teste, law_class)
    if model_aux[1] > model[1]:
        model = model_aux

    return model

In [17]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

models = {}
classes = [ "Administração Pública", 
            "Economia e Desenvolvimento", 
            "Honorífico", 
            "Infraestrutura", 
            "Jurídico", 
            "Meio Ambiente", 
            "Organização do Estado", 
            "Orçamento Público", 
            "Política Social", 
            "Soberania, Defesa Nacional e Ordem Pública"]

for idx in range(len(classes)):
    print(classes[idx])
    models[idx] = model(classes[idx])

Administração Pública
Algorithm: Logistic Regression F1-Score: 0.66
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3331
           1       0.56      0.82      0.66       157

    accuracy                           0.96      3488
   macro avg       0.77      0.90      0.82      3488
weighted avg       0.97      0.96      0.97      3488

Algorithm: Random Forest F1-Score: 0.64
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3331
           1       0.62      0.67      0.64       157

    accuracy                           0.97      3488
   macro avg       0.80      0.82      0.81      3488
weighted avg       0.97      0.97      0.97      3488

Algorithm: Multi-layer Perceptron F1-Score: 0.64
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      3331
           1       0.68      0.60      0.64       157

    accuracy             

In [18]:
len(classes)

10

In [19]:
def predict(model, column_names):
    law_class = model[0]
    accuracy = model[1]
    classifier = model[2]
    vectorizer = model[3]
    classifier_name = model[4]

    predicted_data = leis_nao_classificadas['TXT_EMENTA'].dropna(how='all')
    predicted_data.reset_index(drop=True, inplace=True)

    result = classifier.predict(vectorizer.transform(predicted_data))
    predicted_data.index.name = 'INDEX'   
    file = Path("predicted_law.csv")
   
    if file.is_file():
        predicted_law = pd.read_csv("predicted_law.csv", sep='|', encoding='UTF-8', names=column_names, header=0)
    else:
        predicted_law = pd.DataFrame(predicted_data)
        predicted_law.insert(1, 'PREDICT', "No", True)
        predicted_law.insert(2, 'CLASS_DES_GROUP_1', "Not identified", True)
        predicted_law.insert(3, 'SCORE_GROUP_1', "none", True)
        predicted_law.insert(4, 'ALGORITHM_GROUP_1', "none", True)

    create_columns = 1
    next_group_index1 = predicted_law.shape[1]
    next_group_index2 = next_group_index1 + 1
    next_group_index3 = next_group_index2 + 1
    position = len(column_names)-1
    lenth = len(column_names[position])
    next_goup_number = int(column_names[position][lenth-1:lenth])+1
    for idx in range(len(result)):

        if (result[idx] == 1): 
            if (predicted_law['PREDICT'][idx] == 'No'): 
                predicted_law['PREDICT'][idx] = 'Yes'
                predicted_law['CLASS_DES_GROUP_1'][idx] = law_class
                predicted_law['SCORE_GROUP_1'][idx] = round(accuracy, 2)
                predicted_law['ALGORITHM_GROUP_1'][idx] = classifier_name
            else:
                if (create_columns == 1):
                      create_columns = 0

                      predicted_law.insert(next_group_index1, 'CLASS_DES_GROUP_'+str(next_goup_number), "", True)
                      column_names.append('CLASS_DES_GROUP_'+str(next_goup_number))
                      predicted_law['CLASS_DES_GROUP_'+str(next_goup_number)][idx] = law_class

                      predicted_law.insert(next_group_index2, 'SCORE_GROUP_'+str(next_goup_number), "", True)
                      column_names.append('SCORE_GROUP_'+str(next_goup_number))
                      predicted_law['SCORE_GROUP_'+str(next_goup_number)][idx] = round(accuracy, 2)
                      
                      predicted_law.insert(next_group_index3, 'ALGORITHM_GROUP_'+str(next_goup_number), "", True)
                      column_names.append('ALGORITHM_GROUP_'+str(next_goup_number))
                      predicted_law['ALGORITHM_GROUP_'+str(next_goup_number)][idx] = classifier_name
                else:
                      predicted_law['CLASS_DES_GROUP_'+str(next_goup_number)][idx] = law_class
                      predicted_law['SCORE_GROUP_'+str(next_goup_number)][idx] = round(accuracy, 2)
                      predicted_law['ALGORITHM_GROUP_'+str(next_goup_number)][idx] = classifier_name

    predicted_law.to_csv("predicted_law.csv", sep='|', encoding='UTF-8', header=column_names, index=False)
    print(predicted_law)

    return column_names  

In [20]:
file = 'predicted_law.csv'
column_names = ['TXT_EMENTA', 'PREDICT', 'CLASS_DES_GROUP_1', 'SCORE_GROUP_1', 'ALGORITHM_GROUP_1']
if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)
for idx in range(len(classes)):
    print(classes[idx])
    column_names = predict(models[idx], column_names)

Administração Pública
                                              TXT_EMENTA PREDICT  \
INDEX                                                              
0      Autoriza o Poder Executivo a abrir ao Ministér...      No   
1      Dispõe sobre os vencimentos dos Ministros do S...     Yes   
2      Dispõe sobre a forma e a apresentação dos Símb...      No   
3      Reorganiza e dá nova denominação à Procuradori...      No   
4      Prorroga por doze meses, a contar de 6 (seis) ...      No   
...                                                  ...     ...   
9845   Dispõe sobre o funccionamento da Camara Munici...     Yes   
9846   Concede as subvenções anuais de Cr$ 1.000.000,...      No   
9847   Institui o Sistema Nacional de Armas - SINARM,...      No   
9848   Dispõe sobre a renovação de eleições para Pref...      No   
9849   Determina os casos em que forças estrangeiras ...      No   

           CLASS_DES_GROUP_1 SCORE_GROUP_1    ALGORITHM_GROUP_1  
INDEX                      